In [1]:
import langchain
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

# First steps

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [4]:
#LLM invoke
llm.invoke("Can you explain how large language models work?")

AIMessage(content='Large language models work by using a vast amount of text data to learn patterns and relationships between words. These models are typically trained on massive datasets of text, such as books, articles, and websites, in order to understand the structure and meaning of language.\n\nThe models use a technique called deep learning, which involves passing the text data through multiple layers of interconnected neurons to extract and learn the underlying patterns. These models are trained to predict the next word in a sentence based on the context of the previous words, allowing them to generate coherent and contextually relevant text.\n\nLarge language models can be used for a variety of natural language processing tasks, such as text generation, translation, summarization, and sentiment analysis. They have become increasingly popular in recent years due to their ability to generate human-like text and perform complex language tasks with high accuracy.')

In [5]:
from langchain_core.prompts import ChatPromptTemplate

#Prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a university teacher assistant."),
    ("user", "{input}")
])

#LLM chain
chain = prompt | llm 

chain.invoke({"input": "Can you explain how large language models work?"})

AIMessage(content="Sure! Large language models, such as GPT-3 (Generative Pre-trained Transformer 3), are based on deep learning techniques called transformers. These models are trained on massive amounts of text data to understand and generate human-like text.\n\nHere's a simplified explanation of how large language models work:\n\n1. **Input Encoding**: The model takes an input sequence of text tokens (words or subwords) and converts them into numerical representations using embeddings.\n\n2. **Transformer Architecture**: Large language models use a transformer architecture, which consists of multiple layers of self-attention mechanisms and feedforward neural networks. Self-attention allows the model to weigh the importance of different words in the input sequence when generating the output.\n\n3. **Training**: The model is trained on a large dataset using unsupervised learning techniques. During training, the model learns to predict the next word in a sequence given the previous wor

In [6]:
from langchain_core.output_parsers import StrOutputParser

#String parser to transform aoutput to string
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "Can you explain how large language models work?"})

"Certainly! Large language models, such as GPT-3 (Generative Pre-trained Transformer 3), are based on a type of artificial intelligence called deep learning. These models are designed to understand and generate human language text.\n\nHere's a simplified explanation of how large language models like GPT-3 work:\n\n1. **Training Data**: Large language models are trained on massive amounts of text data from the internet. This data includes books, articles, websites, and other sources of written language. The model learns the patterns and structures of language from this training data.\n\n2. **Transformer Architecture**: Large language models like GPT-3 are built on a transformer architecture, which allows them to process and generate text by considering the context of words and sentences. Transformers use attention mechanisms to focus on different parts of the input text when generating output.\n\n3. **Pre-training and Fine-tuning**: Before being used for specific tasks, large language m

# Retrieval Chain

In [7]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [9]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

#Chain that takes question and already retrieved document and generate answer
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [11]:
from langchain.chains import create_retrieval_chain

#Add document retriever to the chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It provides tracing capabilities and evaluation capabilities to assist with testing LLM applications.


# Conversation Retrieval Chain

In [12]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [13]:
from langchain_core.messages import HumanMessage, AIMessage

#We make up a chat history
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

In [14]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [15]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})['answer']

'LangSmith can help you test your LLM applications by providing tracing and evaluation capabilities. With tracing, you can debug and monitor your chains and intelligent agents built on any LLM framework. The evaluation feature allows you to test and evaluate the performance of your applications. Additionally, LangSmith seamlessly integrates with LangChain, the open source framework for building with LLMs, providing a comprehensive platform for testing and monitoring your LLM applications.'

# Agent

In [16]:
from langchain.tools.retriever import create_retriever_tool
from langchain.retrievers.tavily_search_api import TavilySearchAPIRetriever

#Create retriever tool and search tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

search = TavilySearchAPIRetriever(k=3)

tools = [retriever_tool, search]

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
agent_executor.invoke({"input": "what is the weather in SF?"})


# LangServe

In [ ]:
#AGENT NOT WORKING